# 信号处理仿真与应用-MWorks版-案例

# 第7章 频谱分析

## 7.2 子空间方法

### 7.2.1 频率估计子空间方法

此示例说明如何使用子空间方法解析紧密间隔的正弦波。子空间方法假设谐波模型由加性噪声中的正弦波总和组成，可能是复数。在复值谐波模型中，噪声也是复值的。


##### Julia


创建一个长度为 24 个样本的复值信号。该信号由两个频率为 0.4 Hz 和 0.425 Hz 的复指数（正弦波）和加性复高斯白噪声组成。噪声的均值和方差为零0.22.在复白噪声中，实部和虚部的方差都等于总方差的一半。


In [1]:
using TyPlot
using TySignalProcessing

# Define the signal
n = 0:23
x = exp.(1im*2*pi*0.4*n) .+ exp.(1im*2*pi*0.425*n) .+ 0.2/sqrt(2)*(randn(length(n)) .+ 1im*randn(length(n)))

24-element Vector{ComplexF64}:
   2.0449981930576366 - 0.012383224798065601im
   -1.556166960402212 + 1.1069113099405745im
   0.7693817151546689 - 1.6936539915422621im
  0.04558453904640272 + 1.8733793377121688im
  -0.8346674954394782 - 1.517168637601805im
   1.6114529176090802 + 0.4202900510496829im
  -1.8458385903640677 + 0.2597718378210633im
    1.402620957679915 - 0.8769505188972787im
  -0.5848134450027515 + 1.5930665196094935im
  -0.4273040247484525 - 1.4460581999830582im
                      ⋮
   0.3852646501386019 + 0.38884740373148624im
  -0.6496537642748601 - 0.48848661861223164im
  0.38640855754884834 + 0.11384491614498307im
  -0.3572700423172635 + 0.07808964394550025im
  0.22618068438429537 + 0.11105491073532911im
  -0.2911132073536162 + 0.3186313290131817im
 0.025368492991672903 + 0.10845358074099865im
 -0.10822069708638152 - 0.04563905095359047im
  0.48096751361360623 + 0.00986827671698428im

尝试使用信号的功率谱来解析两个正弦波。将泄漏设置为最大值以获得最佳效果。

In [2]:
# Power Spectrum
segmentLength = 24
psd,w = pwelch(x,segmentLength,nargout=2)
figure()
plot( w/(2*pi), 20*log10.(abs.(psd)))
# frequencies,
# , label="Power Spectrum"
xlabel("Frequency")
ylabel("Power/Frequency")
title("Power Spectrum")

PyObject <objects.mw_text.CTitle object at 0x00000182EDAF0748>

周期图显示0.4 Hz附近的宽峰值。由于周期图的频率分辨率为 1/*N*，因此无法解析两个单独的正弦波，其中 *N* 是信号的长度。在这种情况下，1/*N* 大于两个正弦波的间隔。零填充无助于解析两个单独的峰。

使用子空间方法解析两个紧密间隔的峰。在此示例中，使用 MUSIC 方法。估计自相关矩阵并将自相关矩阵输入到 pmusic 中。指定具有两个正弦分量的模型。绘制结果图。

In [3]:
X,R = corrmtx(x,14,"modified");
Pxx, w, = pmusic(R, 2 ; plotfig=false)
figure()
plot(w/(2*pi), 20 * log10.(abs.(Pxx)))
xlabel("Frequency (Hz)")
ylabel("Power (dB)")
title("Pseudospectrum Estimate via MUSIC")

PyObject <objects.mw_text.CTitle object at 0x00000182BDD04BC8>

MUSIC方法能够分离0.4 Hz和0.425 Hz的两个峰值。但是，子空间方法不会像功率谱密度估计那样产生功率估计。子空间方法对于频率识别最有用，并且对模型顺序错误规范很敏感。